In [12]:
### 1) PREP
######################################################################

# Load necessary packages

import pandas as pd
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import os.path
import os
import numpy as np
from sklearn.linear_model import LinearRegression

# trying p-values
import statsmodels.api as sm
from scipy import stats
import math

# To print colored text
from termcolor import colored

In [13]:
# Set path

path = '/Users/sakshimohan/Dropbox (Personal)/District Allocation Formula/1. Final Documents/Data/input'
os.chdir(path)
#os.getcwd() # check path

In [14]:
# Load dataset

wb = load_workbook('RAF_Final Dataset_v3.xlsm', data_only=True)
sheet = wb['Data for Python']

In [15]:
### 2) DATAFRAME SET-UP
######################################################################

df = pd.DataFrame(sheet.values)
df = df.T # transpose
df.columns = df.iloc[0] # re-index
df = df.reindex(df.index.drop(0))

df = df.iloc[0:38,] # drop empty rows ## V2
df["District"] = df["Variable/District"]
df = df.set_index("Variable/District")

# Drop repeated data (i.e. disaggregated city and district data where the same local government controls the health budget)
df = df.drop("Mzimba", axis=0)
df = df.drop("Blantyre City", axis=0)
df = df.drop("Lilongwe City", axis=0)
df = df.drop("Mzuzu City", axis=0)
df = df.drop("Zomba City", axis=0)
df = df.drop("Blantyre", axis=0)
df = df.drop("Lilongwe", axis=0)
df = df.drop("Mzimba North", axis=0)
df = df.drop("Zomba", axis=0)

df

,Females 15-49,Women age 15-49 currently pregnant (%),Lives births (amended),Population 15-49,Females 10-14 years,Proportion who suffered incidence of sickness in past 2 weeks,Of those who suffered sickness proportion who suffered fever or malaria,Population >15,Population 0-4,Children <5,...,Total population >15,Total population <15,Proportion chronically ill,Of those chronically ill proportion with mental illness,Of those chronically ill proportion with Epilepsy,Population >5,Total EHP Cost,Total EHP Cost (less donor contributions) - using RM program data,Total EHP Cost (less donor contributions) - using RM district data,District
Variable/District,,,,,,,,,,,,,,,,,,,,,
Chitipa,52347,7.6,6052,99209,14230,15.4,32.4,119004,47023,47023,...,119004,115793,3,6,5.5,187774,1.55825e+06,334847,332611,Chitipa
Karonga,85155,8.7,8853,164234,23775,14.2,38.9,196199,67270,67270,...,196199,174172,5,7.7,3.3,303101,3.39464e+06,573116,711319,Karonga
Nkhata Bay,69166,8.2,5466,136726,17400,24.2,46.6,165232,52032,52032,...,165232,131119,5.8,4.7,2.5,244319,2.75016e+06,468815,967195,Nkhata Bay
Rumphi,52650,6.9,4742,104209,13199,20.9,40.6,124836,40266,40266,...,124836,101085,6,3.7,6.5,185655,1.98456e+06,355014,939625,Rumphi
Likoma,2640,6.7,446,5123,667,18.9,28.7,6209,1620,1620,...,6209,4285,7.9,8.8,5.3,8874,100550,18646.8,20999.8,Likoma
Kasungu,203948,6.8,19573,407137,56970,30.5,35.7,477107,182597,182597,...,477107,450437,5.4,6.2,3,744947,7.17356e+06,1.33351e+06,2.13013e+06,Kasungu
Nkhotakota,93646,7.6,11923,185306,26182,19.7,47.4,219550,78570,78570,...,219550,197523,4.7,1.2,3.7,338503,3.60211e+06,612358,1.04579e+06,Nkhotakota
Ntchisi,67800,9.1,7641,133293,19706,5.8,44.9,158415,63857,63857,...,158415,157475,5.1,3.7,10.6,252033,2.0034e+06,454285,1.15572e+06,Ntchisi
Dowa,194982,5.9,17256,385446,48593,32.9,41.3,459614,167897,167897,...,459614,406600,5.3,5.2,3.7,698317,6.73101e+06,1.30905e+06,2.59794e+06,Dowa


In [16]:
## Generate dataframes consisting of independent variables and dependent variable 

x = df.dropna(how='all')
x = x.drop('Total EHP Cost',1)
x = x.drop('District',1) # Variable/District under V1
#x = x.drop('Total EHP Cost (less donor contributions)',1)
#x = x.drop('Total EHP Cost (HIV prevalence adjusted)',1)
x = x.drop('Total EHP Cost (less donor contributions) - using RM program data',1)
x = x.drop('Total EHP Cost (less donor contributions) - using RM district data',1)

y = df['Total EHP Cost']
y = y.dropna()
y_lessdonor = df['Total EHP Cost (less donor contributions) - using RM program data']
y_lessdonor = y_lessdonor.dropna()
y_lessdonor_rmdist = df['Total EHP Cost (less donor contributions) - using RM district data']
y_lessdonor_rmdist = y_lessdonor_rmdist.dropna()

## Convert values to numeric
x = x.apply(pd.to_numeric, errors = 'coerce')
y = y.apply(pd.to_numeric, errors = 'coerce')
y_lessdonor = y_lessdonor.apply(pd.to_numeric, errors = 'coerce')
y_lessdonor_rmdist = y_lessdonor_rmdist.apply(pd.to_numeric, errors = 'coerce')
x.describe()
for col in x.columns:
    print(col)

Females 15-49
Women age 15-49 currently pregnant (%)
Lives births (amended)
Population 15-49
Females 10-14 years
Proportion who suffered incidence of sickness in past 2 weeks
Of those who suffered sickness proportion who suffered fever or malaria
Population >15
Population 0-4
Children <5
Propotion of children <5 -3 SD below weight-for-age
Children <15
Proportion of children aged 6-23 months fed minimum dietary diversity
HIV prevalence among 15-49 (%)
Total Population
Population >40
Total population >15
Total population <15
Proportion chronically ill
Of those chronically ill proportion with mental illness
Of those chronically ill proportion with Epilepsy
Population >5


In [17]:
## Validation
    # Total EHP Cost (HIV Prevalence adjusted) = 185,164,299
    # Total EHP Cost (less donor contributions) (HIV Prevalence adjusted) = 53,653,998
print(y.sum(skipna = True), "should be equal to 185,164,299")
print(y_lessdonor.sum(skipna = True), "should be equal to 53,653,998") 
if (-2 < y.sum(skipna = True) - 185164299 < 2) & (-2 < y_lessdonor.sum(skipna = True)-53653998 <2):
    print("VERIFIED")
else:
    print("ERROR IN DATA")

185164299.70839068 should be equal to 185,164,299
27579087.181493983 should be equal to 53,653,998
ERROR IN DATA


In [18]:
### 3) DESCRIPTIVE ANALYSIS
######################################################################

## CORRELATION BETWEEN INDEPENDENT VARIABLES

# Re-order columns for correlation matrix:
columnsTitles = ['Total Population', 'Population >40', 'Females 15-49','Population 15-49',
                 'Total population >15','Population >15','Total population <15','Children <15','Females 10-14 years',
                 'Population >5','Population 0-4','Children <5','Lives births (amended)','Women age 15-49 currently pregnant (%)',
                 'Proportion who suffered incidence of sickness in past 2 weeks','Of those who suffered sickness proportion who suffered fever or malaria',
                 'Propotion of children <5 -3 SD below weight-for-age','Proportion of children aged 6-23 months fed minimum dietary diversity',
                 'HIV prevalence among 15-49 (%)','Proportion chronically ill','Of those chronically ill proportion with mental illness',
                 'Of those chronically ill proportion with Epilepsy',]
x = x.reindex(columns=columnsTitles)

#Store correlation matrix in an excel workbook
corr = x.corr()
(corr.style.background_gradient(cmap='coolwarm')).set_precision(2).to_excel("corr_matrix.xlsx", engine="openpyxl")

# Print correlation matrix
x.corr()
(corr.style.background_gradient(cmap='coolwarm')).set_precision(2)

0,Total Population,Population >40,Females 15-49,Population 15-49,Total population >15,Population >15,Total population <15,Children <15,Females 10-14 years,Population >5,Population 0-4,Children <5,Lives births (amended),Women age 15-49 currently pregnant (%),Proportion who suffered incidence of sickness in past 2 weeks,Of those who suffered sickness proportion who suffered fever or malaria,Propotion of children <5 -3 SD below weight-for-age,Proportion of children aged 6-23 months fed minimum dietary diversity,HIV prevalence among 15-49 (%),Proportion chronically ill,Of those chronically ill proportion with mental illness,Of those chronically ill proportion with Epilepsy
0,,,,,,,,,,,,,,,,,,,,,,
Total Population,1,1,1,1,1,1,1,1,1,1,1,1,0.96,-0.027,0.18,0.13,-0.06,0.22,0.092,0.07,0.024,0.029
Population >40,1,1,1,1,1,1,0.99,0.99,1,1,0.99,0.99,0.95,-0.05,0.2,0.14,-0.047,0.22,0.12,0.099,0.009,0.01
Females 15-49,1,1,1,1,1,1,0.99,0.99,0.99,1,0.99,0.99,0.95,-0.048,0.16,0.14,-0.049,0.23,0.13,0.083,0.014,0.013
Population 15-49,1,1,1,1,1,1,0.99,0.99,0.99,1,0.99,0.99,0.94,-0.059,0.16,0.13,-0.054,0.24,0.12,0.077,0.019,0.011
Total population >15,1,1,1,1,1,1,0.99,0.99,0.99,1,0.99,0.99,0.95,-0.055,0.17,0.14,-0.052,0.23,0.12,0.083,0.017,0.013
Population >15,1,1,1,1,1,1,0.99,0.99,0.99,1,0.99,0.99,0.95,-0.055,0.17,0.14,-0.052,0.23,0.12,0.083,0.017,0.013
Total population <15,1,0.99,0.99,0.99,0.99,0.99,1,1,1,1,1,1,0.97,0.0078,0.2,0.13,-0.068,0.2,0.063,0.053,0.033,0.049
Children <15,1,0.99,0.99,0.99,0.99,0.99,1,1,1,1,1,1,0.97,0.0078,0.2,0.13,-0.068,0.2,0.063,0.053,0.033,0.049
Females 10-14 years,1,1,0.99,0.99,0.99,0.99,1,1,1,1,0.99,0.99,0.96,0.015,0.18,0.14,-0.044,0.18,0.12,0.088,0.0047,0.056


In [19]:
# Based on the correlation matrix, drop superfluous population variables which are highly correlated with total population

x_totpop = x.drop('Females 15-49',1)
x_totpop = x_totpop.drop('Population 15-49',1)
x_totpop = x_totpop.drop('Females 10-14 years',1)
x_totpop = x_totpop.drop('Population >15',1)
x_totpop = x_totpop.drop('Population 0-4',1)
x_totpop = x_totpop.drop('Children <5',1)
x_totpop = x_totpop.drop('Children <15',1)
x_totpop = x_totpop.drop('Population >40',1)
x_totpop = x_totpop.drop('Total population >15',1)
x_totpop = x_totpop.drop('Total population <15',1)
x_totpop = x_totpop.drop('Population >5',1)
x_totpop = x_totpop.drop('Lives births (amended)',1)

In [20]:
### 4) PREPRATION FOR DIMENSION REDUCTION
######################################################################

## Log values

# Indenpendent variables
x_totpop_ln = x_totpop.copy()
for col in x_totpop.columns:
    x_totpop_ln[col] = np.log(x_totpop_ln[col])
    
# Dependent variables
y_ln = y.copy()
y_ln = np.log(y_ln)

y_lessdonor_ln = y_lessdonor.copy()
y_lessdonor_ln = np.log(y_lessdonor_ln)

y_lessdonor_rmdist_ln = y_lessdonor_rmdist.copy()
y_lessdonor_rmdist_ln = np.log(y_lessdonor_rmdist_ln)

In [73]:
### 5) DIMENSION REDUCTION
######################################################################

## 5.i) Recursive feature selection
######################################################################
# Reference: https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# Calculate the optimum number of features in the total EHP cost model
######################################################################
nof_list=np.arange(1,11)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(x_totpop_ln,y_ln, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features (Total EHP Cost): %d" %nof)
print("Score with %d features (Total EHP Cost): %f" % (nof, high_score))

# Calculate the optimum number of features in the total EHP cost less donor contributions model
###############################################################################################
nof_list=np.arange(1,11)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(x_totpop_ln,y_lessdonor_ln, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features (Total EHP Cost less donors): %d" %nof)
print("Score with %d features (Total EHP Cost less donors): %f" % (nof, high_score))

# Calculate the optimum number of features in the total EHP cost less donor contributions model
###############################################################################################
nof_list=np.arange(1,11)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(x_totpop_ln,y_lessdonor_rmdist_ln, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features (Total EHP Cost less donors (using RM district data)): %d" %nof)
print("Score with %d features (Total EHP Cost less donors(using RM district data)): %f" % (nof, high_score))

Optimum number of features (Total EHP Cost): 3
Score with 3 features (Total EHP Cost): 0.993261
Optimum number of features (Total EHP Cost less donors): 2
Score with 2 features (Total EHP Cost less donors): 0.997726
Optimum number of features (Total EHP Cost less donors (using RM district data)): 7
Score with 7 features (Total EHP Cost less donors(using RM district data)): 0.874859


In [74]:
## 5.ii) Backward Elimination based on p-value
######################################################################

# Find an efficient set of features in the total EHP cost model
######################################################################
cols = list(x_totpop_ln.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x_totpop_ln[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_ln,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print("Selected features (Total EHP Cost)", selected_features_BE)

# Find an efficient set of features in the total EHP cost less donor contributions model
###############################################################################################
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x_totpop_ln[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_lessdonor_ln,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print("Selected features (Total EHP Cost less donor contributions) (using RM program data)", selected_features_BE)

# Find an efficient set of features in the total EHP cost less donor contributions model
###############################################################################################
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x_totpop_ln[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_lessdonor_rmdist_ln,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print("Selected features (Total EHP Cost less donor contributions) (using RM district data)", selected_features_BE)

Selected features (Total EHP Cost) ['Total Population', 'Proportion who suffered incidence of sickness in past 2 weeks', 'HIV prevalence among 15-49 (%)', 'Proportion chronically ill']
Selected features (Total EHP Cost less donor contributions) (using RM program data) ['Total Population', 'Proportion chronically ill']
Selected features (Total EHP Cost less donor contributions) (using RM district data) ['Total Population']


In [75]:
## 5.iii) Backward Elimination based on AIC (Constant introduced in exogeneous matrix)
######################################################################

## Total EHP Cost Model
######################################################################
true_values = y_ln
cols = list(x_totpop_ln.columns)
pmax = 1
while (len(cols)>0):
    p= []
    
    X_1 = x_totpop_ln[cols]
    X_1 = sm.add_constant(X_1)
    k = np.shape(X_1)[1]
    n_obs = np.shape(X_1)[0]
    model = sm.OLS(true_values,X_1).fit()
    
    p = pd.Series(model.pvalues.values[1:],index = cols)   
    r2 = model.rsquared
    pmax = max(p)
    feature_with_p_max = p.idxmax()

    predicted_values = model.predict()
    sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
    aic_old = n_obs*math.log(sumofresiduals/n_obs) + 2*k
    
    aic_new = aic_old
    iter = 0
    
    while (aic_new <= aic_old):
        iter = iter + 1 
        print('Iteration',iter)
        
        aic_old = aic_new
        cols.remove(feature_with_p_max)
        print("Variable dropped:", feature_with_p_max,"; p-value of variable:", pmax)
        print("OLD AIC:", aic_old)
        
        X_1 = x_totpop_ln[cols]
        X_1 = sm.add_constant(X_1)
        k = np.shape(X_1)[1]
        print("Degrees of freedom:", k)
        model = sm.OLS(true_values,X_1).fit()

        p = pd.Series(model.pvalues.values[1:],index = cols)   
        r2 = model.rsquared
        pmax = max(p)
        feature_with_p_max = p.idxmax()

        predicted_values = model.predict()
        sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
        aic_new = n_obs*math.log(sumofresiduals/n_obs) + 2*k
        print("NEW AIC:", aic_new)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print(model.summary())

## Total EHP Cost less donor contributions Model
######################################################################
true_values = y_lessdonor_ln
cols = list(x_totpop_ln.columns)
pmax = 1
while (len(cols)>0):
    p= []
    
    X_1 = x_totpop_ln[cols]
    X_1 = sm.add_constant(X_1)
    k = np.shape(X_1)[1]
    n_obs = np.shape(X_1)[0]
    model = sm.OLS(true_values,X_1).fit()

    p = pd.Series(model.pvalues.values[1:],index = cols)   
    r2 = model.rsquared
    pmax = max(p)
    feature_with_p_max = p.idxmax()

    predicted_values = model.predict()
    sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
    aic_old = n_obs*math.log(sumofresiduals/n_obs) + 2*k
    
    aic_new = aic_old
    iter = 0
    
    while (aic_new <= aic_old):
        iter = iter + 1 
        print('Iteration',iter)
        
        aic_old = aic_new
        cols.remove(feature_with_p_max)
        print("Variable dropped:", feature_with_p_max,"; p-value of variable:", pmax)
        print("OLD AIC:", aic_old)
        
        X_1 = x_totpop_ln[cols]
        X_1 = sm.add_constant(X_1)
        k = np.shape(X_1)[1]
        print("Degrees of freedom:", k)
        model = sm.OLS(true_values,X_1).fit()

        p = pd.Series(model.pvalues.values[1:],index = cols)   
        r2 = model.rsquared
        pmax = max(p)
        feature_with_p_max = p.idxmax()

        predicted_values = model.predict()
        sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
        aic_new = n_obs*math.log(sumofresiduals/n_obs) + 2*k
        print("NEW AIC:", aic_new)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print(model.summary())

Iteration 1
Variable dropped: Of those chronically ill proportion with Epilepsy ; p-value of variable: 0.9281424900075858
OLD AIC: -141.36791556594324
Degrees of freedom: 10
NEW AIC: -143.3583910621089
Iteration 2
Variable dropped: Propotion of children <5 -3 SD below weight-for-age ; p-value of variable: 0.8935774177887011
OLD AIC: -143.3583910621089
Degrees of freedom: 9
NEW AIC: -145.25397696825976
Iteration 3
Variable dropped: Of those chronically ill proportion with mental illness ; p-value of variable: 0.401371402676477
OLD AIC: -145.25397696825976
Degrees of freedom: 8
NEW AIC: -145.53688309412516
Iteration 4
Variable dropped: Women age 15-49 currently pregnant (%) ; p-value of variable: 0.2338881422926432
OLD AIC: -145.53688309412516
Degrees of freedom: 7
NEW AIC: -145.3228878164737
['Total Population', 'Proportion who suffered incidence of sickness in past 2 weeks', 'Of those who suffered sickness proportion who suffered fever or malaria', 'Proportion of children aged 6-23 mon

In [76]:
## 5.iii) Backward Elimination based on AIC (No constant in exogeneous matrix + Robust Model)

## Currently ERROR : becuase the pvalue array does not seem to be stored after running RLM
######################################################################


## Total EHP Cost Model
######################################################################
true_values = y_ln
cols = list(x_totpop_ln.columns)
pmax = 1
while (len(cols)>0):
    p= []
    
    X_1 = x_totpop_ln[cols]
    #X_1 = sm.add_constant(X_1)
    k = np.shape(X_1)[1]
    n_obs = np.shape(X_1)[0]
    model = sm.OLS(true_values,X_1).fit()
    #model = sm.RLM(true_values, X_1, M=sm.robust.norms.HuberT())

    p = pd.Series(model.pvalues.values[0:],index = cols)   
    r2 = model.rsquared
    pmax = max(p)
    feature_with_p_max = p.idxmax()

    predicted_values = model.predict()
    sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
    aic_old = n_obs*math.log(sumofresiduals/n_obs) + 2*k
    
    aic_new = aic_old
    iter = 0
    
    while (aic_new <= aic_old):
        iter = iter + 1 
        print('Iteration',iter)
        
        aic_old = aic_new
        cols.remove(feature_with_p_max)
        print("Variable dropped:", feature_with_p_max,"; p-value of variable:", pmax)
        print("OLD AIC:", aic_old)
        
        X_1 = x_totpop_ln[cols]
        #X_1 = sm.add_constant(X_1)
        k = np.shape(X_1)[1]
        print("Degrees of freedom:", k)
        model = sm.OLS(true_values,X_1).fit()
        #model = sm.RLM(true_values, X_1, M=sm.robust.norms.HuberT())

        p = pd.Series(model.pvalues.values[0:],index = cols)   
        r2 = model.rsquared
        pmax = max(p)
        feature_with_p_max = p.idxmax()

        predicted_values = model.predict()
        sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
        aic_new = n_obs*math.log(sumofresiduals/n_obs) + 2*k
        print("NEW AIC:", aic_new)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print(model.summary())

## Total EHP Cost less donor contributions Model
######################################################################
true_values = y_lessdonor_ln
cols = list(x_totpop_ln.columns)
pmax = 1
while (len(cols)>0):
    p= []
    
    X_1 = x_totpop_ln[cols]
    #X_1 = sm.add_constant(X_1)
    k = np.shape(X_1)[1]
    n_obs = np.shape(X_1)[0]
    model = sm.OLS(true_values,X_1).fit()
    #model = sm.RLM(true_values, X_1, M=sm.robust.norms.HuberT())

    p = pd.Series(model.pvalues.values[0:],index = cols)   
    r2 = model.rsquared
    pmax = max(p)
    feature_with_p_max = p.idxmax()

    predicted_values = model.predict()
    sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
    aic_old = n_obs*math.log(sumofresiduals/n_obs) + 2*k
    
    aic_new = aic_old
    iter = 0
    
    while (aic_new <= aic_old):
        iter = iter + 1 
        print('Iteration',iter)
        
        aic_old = aic_new
        cols.remove(feature_with_p_max)
        print("Variable dropped:", feature_with_p_max,"; p-value of variable:", pmax)
        print("OLD AIC:", aic_old)
        
        X_1 = x_totpop_ln[cols]
        #X_1 = sm.add_constant(X_1)
        k = np.shape(X_1)[1]
        print("Degrees of freedom:", k)
        model = sm.OLS(true_values,X_1).fit()
        #model = sm.RLM(true_values, X_1, M=sm.robust.norms.HuberT())

        p = pd.Series(model.pvalues.values[0:],index = cols)   
        r2 = model.rsquared
        pmax = max(p)
        feature_with_p_max = p.idxmax()

        predicted_values = model.predict()
        sumofresiduals = sum(np.array([(predicted_values[xi]-true_values[xi])**2 for xi in range(1,n_obs)]))
        aic_new = n_obs*math.log(sumofresiduals/n_obs) + 2*k
        print("NEW AIC:", aic_new)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)
print(model.summary())

Iteration 1
Variable dropped: Propotion of children <5 -3 SD below weight-for-age ; p-value of variable: 0.91595570494512
OLD AIC: -142.9343789997433
Degrees of freedom: 9
NEW AIC: -144.8312007501951
Iteration 2
Variable dropped: Of those chronically ill proportion with Epilepsy ; p-value of variable: 0.8193161410882223
OLD AIC: -144.8312007501951
Degrees of freedom: 8
NEW AIC: -146.78395448431425
Iteration 3
Variable dropped: Women age 15-49 currently pregnant (%) ; p-value of variable: 0.3265506589006544
OLD AIC: -146.78395448431425
Degrees of freedom: 7
NEW AIC: -146.5843863550774
['Total Population', 'Proportion who suffered incidence of sickness in past 2 weeks', 'Of those who suffered sickness proportion who suffered fever or malaria', 'Proportion of children aged 6-23 months fed minimum dietary diversity', 'HIV prevalence among 15-49 (%)', 'Proportion chronically ill', 'Of those chronically ill proportion with mental illness']
                            OLS Regression Results  